# aws identity access management (IAM)

I like to say that if you're having a problem with aws, it's probably going to get fixed on the `iam` console.

the most common problem area for new users of cloud services -- and *especially* aws -- is permissions and security. in fact, the task can be confusing and daunting enough that entire companies have sprung up offering to take care of these issues for a fee. given that this is already a full time job for experienced IT staffers in every organization with shared resources (*e.g.* any company or university), this is perhaps not surprising. 

It's easy to take that role for granted. I think learning some *basics* about what they do will not only allow you to appreciate what they're doing a bit more, it will give you the ability to think a bit more like they do, to diagnose and solve problems like they do, and to help communicate with them when help is needed.

plus, you just have to figure it out. it may get easier over time, but that doesn't matter right now!

with that set of "this is hard" caveats out of the way, you should know: the basics of user control are time-tested and well organized and in aws, generally speaking, the defaults work.

aws has already gone to great lengths to make sure that everything you do in the cloud is as secure as possible -- that is, everything is locked down by default. our job is to learn how to *unlock* down... or is it to lock *up*... or...

nevermind.

## semantics and relationships

permissions structures usually exist as *people* who belong to *groups* that can *do things*. the language changes from application to application, so let's lay down a couple of the main terms and the relationships between them:

1. single people and things are **`users`**
2. **`users`** belong to **`groups`**
3. **`users`** and **`groups`** of **`users`** can have **`roles`**
4. **`users`**, **`groups`**, and **`roles`** can possess **`permissions`** or **`policies`**
5. **`policies`** are collections of **`permissions`**

you're a sysad now, right?

maybe some more details will help.

open the [`iam` console](https://console.aws.amazon.com/iam/home?region=us-east-1) for your aws account

## users

a `user` is an inidivdual account for your aws instance. although we typically think of a `user` as being a living person, the idea is generalized to include any independent entity which could do something (or be prevented from doing something):

1. people who log in to machines and do things
2. accounts you create to run specific jobs
3. system accounts that need special elevated priveleges
4. other

the end goal is to give every user permission to do the things they need to do -- and *only* those permissions.

in our initial account setup, we created a user for our personal aws use. let's look through the properties of that user object.

1. open [the `iam` users dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/users) ("Users" in the left menu)
2. click the link in the "User name" column of the table

1. permissions tab
    1. the summary page starts off with the permission tab active
    2. permissions that our special user has are collected here as "policies" (more below).
    3. users can get permissions directly, or via group membership
    4. the “policy type” field will explain the source of a given permission policy.
    5. click the drop-down on any of the attached policies to see the full list of permissions this user has.
2. groups tab
    1. the "groups" tab displays all the groups to which your user belongs
3. security credentials tab
    1. any user can be granted an ssh key for ssh access to ec2 instances
    2. Access keys are created for API access to various services (see aws cli below)
4. access advisor tab
    1. for more advanced usage and creating access policies

## groups

groups are simply collections of users (and other groups).

We create groups to create "permission templates." Organizationally, you will usually have large groups of people that do just about the same thing, and need just about the same permissions. Rather than copy/paste the same permissions to every new user, you create a group for that "type" or user (*e.g.* "data scientists") and give the *group* those permissions.

this means that spinning up "data scientist caitlin" is easier, because you can just say "CAITLIN is a DATA SCIENTIST"

click now on [the `iam` groups dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/groups) ("Groups" in the left menu).

this should bring up a table with your groups in them. click on one of your groups and you will land on a summary page for that group, which contains the following tabs:

1. users tab
    1. a list of the users in this group
    2. add or remove any number of users from this group -- note: this is instantaneous
2. permissions tab
    1. a list of the policies that are attached to this group (no drop-down like with users)
3. access advisor tab
    1. same as above for users -- advanced dialog for designing access policies

## roles

a role is similar to a user in that it is a "single entity" with specific purposes and abilities, but it is not tied to a real-world user or a single process or account. instead, it's a position that a `user` could step in to.

a metaphor: the position of `hall monitor` (a `role`) has certain priveleges and responsibilities, but `kids` (`users`) take turns stepping into that role. while a `kid` is the `hall monitor` they can do special things; and if a `kid` tried to issue a tardy slip while not being `hall monitor` it would not go well.

click now on [the `iam` roles dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/roles) ("Roles" in the left menu).

this should bring up a table with your roles listed there (you may have none). if you do, click on one of your roles and you will land on a summary page for that role, which contains the following tabs:

1. permissions tab
    1. a list of the policies that are attached to this group (no drop-down like with users)
2. trust relationships tab
    1. often organizations will have external user management systems (e.g. windows active directory). "trust relationships" are statements that what those external user systems say should be trusted
3. access advisor tab
    1. same as above for users -- advanced dialog for designing access policies
4. revoke sessions
    1. an emergency lever to restrict access to this role and its associated permissions

## policies

a policy is a collection of related permissions. these are collections like "AmazonS3ReadOnlyAccess", which holds every permission needed to read files on `s3` but no further.

amazon cultivates many of these collections of permissions, and usually the ones you need are

1. named in an obvious way, and
2. added to the desired accounts as you set up the services

click now on [the `iam` policies dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/policies) ("Policies" in the left menu).

this should bring up a table with thousands of policies (mostly created by AWS and not you). click on any one of them and you will land on a summary page for that policy, which contains the following tabs:

1. permissions tab
    1. unlike the previous tabs, this permissions tab lists real *permissions*. see below for more
2. attached entites tab
    1. the users, groups, or roles that have this policy attached to them
3. policy versions
    1. a versioned (numbered) history of the permission json statement
4. access advisor tab
    1. same as above -- advanced dialog for designing access policies

## permissions

A permission is a codification of an access to or restriction from specific service. each permission consists of four pieces:

1. an aws service (e.g. ec2, s3, rds)
2. an access level (read, write, etc)
3. a resource
    1. if you have multiple resources (like hundreds of `s3` buckets) in your organization, this allows you indicate which instance(s) of those services this rule applies to
4. request condition
    1. a paramaterizable prerequisite for exercising this permission


## other

the remaining `iam` tabs are all useful in certain contexts, but we won't dive into them right now.

1. identity providers: allows you to "trust" outside resources for identity authentication
2. account settings: general security settings
3. credential report: full automated export of "who" can do "what"
4. encryption keys: centralized resource for doing message encryption between your services

<div align="center">**mini exercise: let's create a `gu511` user**</div>

1. go to the `users` dashboard for the `iam` service
2. create a user ("Add user")
    1. call this user `gu511`
    2. give this user *programmatic access only*
    3. create a group `s3_full_access`
        1. find the `AmazonS3FullAccess` policy and add it
        2. find the `AmazonEC2FullAccess` policy and add it
    4. put `gu511` in this group

***<div align="center">UNLEASH THE POWER</div>***
<div align="center"><img src="https://www.linux.com/sites/lcom/files/styles/rendered_file/public/workstation-security_0.jpg?itok=g907Js22"></div>

# END OF LECTURE

next lecture: [AWS programmatic interfaces](009_cli_boto.ipynb)